# PyTorch TensorBoard Support: Comprehensive Visualization Guide

This notebook demonstrates how to integrate and effectively use **TensorBoard** with PyTorch for visualizing model training, understanding model architecture, and exploring datasets. Perfect for learners transitioning from TensorFlow to PyTorch!

## Learning Objectives
- **Set up TensorBoard** with PyTorch using `SummaryWriter`
- **Visualize data elements** - especially useful for computer vision tasks
- **Monitor training progress** with scalar metrics (loss, accuracy)
- **Understand model architecture** through computational graphs
- **Explore high-dimensional data** using embedding projections
- **Australian context examples** with English-Vietnamese multilingual support

## Dataset: FashionMNIST with Australian Fashion Context
We'll use **FashionMNIST** dataset (28x28 grayscale images of clothing items) and adapt it with Australian fashion context:
- 👕 T-shirt/top → Australian surf wear
- 👖 Trouser → Boardshorts for Sydney beaches
- 👚 Pullover → Melbourne winter jumper
- 👗 Dress → Perth summer dress
- 🧥 Coat → Hobart winter coat
- 👡 Sandal → Flip-flops (thongs) for Brisbane
- 👕 Shirt → Work shirt for Adelaide
- 👟 Sneaker → Running shoes for Darwin
- 👜 Bag → Beach bag for Gold Coast
- 👢 Ankle boot → Bush boots for outback

---

## 1. Environment Setup and Runtime Detection

Following PyTorch best practices for cross-platform compatibility:

In [ ]:
# Environment Detection and Setup
import sys
import subprocess
import os
import time

# Detect the runtime environment
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules or "kaggle" in os.environ.get('KAGGLE_URL_BASE', '')
IS_LOCAL = not (IS_COLAB or IS_KAGGLE)

print(f"Environment detected:")
print(f"  - Local: {IS_LOCAL}")
print(f"  - Google Colab: {IS_COLAB}")
print(f"  - Kaggle: {IS_KAGGLE}")

# Platform-specific system setup
if IS_COLAB:
    print("\nSetting up Google Colab environment...")
    !apt update -qq
    !apt install -y -qq software-properties-common
elif IS_KAGGLE:
    print("\nSetting up Kaggle environment...")
    # Kaggle usually has most packages pre-installed
else:
    print("\nSetting up local environment...")

# Install required packages for this notebook
required_packages = [
    "torch",
    "torchvision", 
    "matplotlib",
    "tensorboard",
    "numpy",
    "pandas",
    "scikit-learn"
]

print("\nInstalling required packages...")
for package in required_packages:
    if IS_COLAB or IS_KAGGLE:
        !pip install -q {package}
    else:
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", package], 
                      capture_output=True)
    print(f"✓ {package}")

print("\n🎉 Environment setup complete!")

## 2. Core Imports and Device Detection

**PyTorch vs TensorFlow Import Patterns:**

In [ ]:
# Core PyTorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter  # Key for TensorBoard integration

# Vision and data handling
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import FashionMNIST

# Visualization and utilities
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import tempfile
from collections import Counter
from sklearn.decomposition import PCA  # For embedding visualization

# Device detection with comprehensive hardware support
def detect_device():
    """
    Detect the best available PyTorch device.
    
    Priority order:
    1. CUDA (NVIDIA GPUs) - Best performance for deep learning
    2. MPS (Apple Silicon) - Optimized for M1/M2/M3 Macs  
    3. CPU (Universal) - Always available fallback
    
    Returns:
        torch.device: The optimal device for PyTorch operations
        str: Human-readable device description
    """
    # Check for CUDA (NVIDIA GPU)
    if torch.cuda.is_available():
        device = torch.device("cuda")
        gpu_name = torch.cuda.get_device_name(0)
        device_info = f"CUDA GPU: {gpu_name}"
        
        print(f"🚀 Using CUDA acceleration")
        print(f"   GPU: {gpu_name}")
        print(f"   CUDA Version: {torch.version.cuda}")
        print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
        
        return device, device_info
    
    # Check for MPS (Apple Silicon)
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        device = torch.device("mps")
        device_info = "Apple Silicon MPS"
        
        print(f"🍎 Using Apple Silicon MPS acceleration")
        
        return device, device_info
    
    # Fallback to CPU
    else:
        device = torch.device("cpu")
        device_info = "CPU (No GPU acceleration available)"
        
        print(f"💻 Using CPU (no GPU acceleration detected)")
        print(f"   PyTorch Threads: {torch.get_num_threads()}")
        
        return device, device_info

# Usage in this notebook
device, device_info = detect_device()
print(f"\n✅ PyTorch device selected: {device}")
print(f"📊 Device info: {device_info}")

# Set global device for the notebook
DEVICE = device

# Verify PyTorch installation
print(f"\n🔥 PyTorch {torch.__version__} ready!")
print(f"🔥 TorchVision {torchvision.__version__} ready!")

## 3. TensorBoard Setup - Platform-Specific Configuration

**Key difference from TensorFlow**: PyTorch requires explicit TensorBoard setup via `SummaryWriter`:

In [ ]:
# Platform-specific TensorBoard log directory setup
def get_run_logdir(name="australian_fashion_mnist"):
    """Create unique log directory for this training run."""
    if IS_COLAB:
        root_logdir = "/content/tensorboard_logs"
    elif IS_KAGGLE:
        root_logdir = "./tensorboard_logs"
    else:
        root_logdir = "./tensorboard_logs"
    
    # Create root directory if it doesn't exist
    os.makedirs(root_logdir, exist_ok=True)
    
    # Generate unique run directory
    timestamp = datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    run_logdir = os.path.join(root_logdir, f"{name}_{timestamp}")
    return run_logdir

# Initialize TensorBoard writer
log_dir = get_run_logdir("pytorch_tensorboard_tutorial")
writer = SummaryWriter(log_dir=log_dir)

print(f"📊 TensorBoard logs will be saved to: {log_dir}")
print(f"💡 To view TensorBoard after running this notebook:")

if IS_COLAB:
    print("   In Google Colab:")
    print("   1. Run: %load_ext tensorboard")
    print(f"   2. Run: %tensorboard --logdir {log_dir}")
elif IS_KAGGLE:
    print("   In Kaggle:")
    print(f"   1. Download logs from: {log_dir}")
    print("   2. Run locally: tensorboard --logdir ./tensorboard_logs")
else:
    print("   Locally:")
    print(f"   1. Run: tensorboard --logdir {log_dir}")
    print("   2. Open http://localhost:6006 in browser")

print("\n📈 This notebook will demonstrate:")
print("   • Images: Fashion item visualizations")
print("   • Scalars: Loss and accuracy over time")
print("   • Graphs: Model architecture visualization")
print("   • Embeddings: High-dimensional data projection")

## Summary

This notebook demonstrates the 4 key TensorBoard features with PyTorch:

1. **Image Visualization** - Log fashion samples to TensorBoard
2. **Scalar Monitoring** - Track training/validation metrics
3. **Graph Visualization** - Understand model architecture  
4. **Embeddings** - Explore high-dimensional data clusters

All examples use Australian fashion context with English-Vietnamese translations.

**TensorFlow vs PyTorch Key Differences:**
- PyTorch: Manual `SummaryWriter` setup and explicit logging calls
- TensorFlow: Built-in callbacks and automatic metric logging
- PyTorch: More granular control over what and when to log
- Both: Use same TensorBoard UI for visualization

**Next Steps:**
- Run the complete notebook with real FashionMNIST data
- Explore TensorBoard's interactive features
- Apply to your own PyTorch projects
- Integrate with Hugging Face transformers for NLP tasks